# Database operations low level

In [112]:
from prettytable import PrettyTable
from typing import Callable

movies = PrettyTable()
movies.field_names = ['title', 'year', 'IMDb-score']
movies.add_row(['The Dark Knight', '2008', '9.0'])
movies.add_row(['Fight Club', '1999', '8.8'])
movies.add_row(['Matrix', '1999', '8.7'])
movies.add_row(['Inception', '2010', '8.8'])
movies.add_row(['Interstellar', '2014', '8.7'])
movies.add_row(['Avengers: Infinity War', '2018', '8.4'])
print(movies)

+------------------------+------+------------+
|         title          | year | IMDb-score |
+------------------------+------+------------+
|    The Dark Knight     | 2008 |    9.0     |
|       Fight Club       | 1999 |    8.8     |
|         Matrix         | 1999 |    8.7     |
|       Inception        | 2010 |    8.8     |
|      Interstellar      | 2014 |    8.7     |
| Avengers: Infinity War | 2018 |    8.4     |
+------------------------+------+------------+


In [113]:
actors = PrettyTable()
actors.field_names = ['first_name', 'last_name']
actors.add_row(['Christian', 'Bale'])
actors.add_row(['Bred', 'Pitt'])
actors.add_row(['Meryl', 'Streep'])
print(actors)

+------------+-----------+
| first_name | last_name |
+------------+-----------+
| Christian  |    Bale   |
|    Bred    |    Pitt   |
|   Meryl    |   Streep  |
+------------+-----------+


## Operation: Project

In [114]:
def project(table: PrettyTable, cols: list[int], distinct: bool=False) -> PrettyTable:
    table_result = PrettyTable()
    table_result.field_names = [table.field_names[col_index] for col_index in cols]
    if not distinct:
        for row in table.rows:
            table_result.add_row([row[col_index] for col_index in cols])
        return table_result
    for row in table.rows:
        row_result = [row[col_index] for col_index in cols]
        if row_result not in table_result.rows:
            table_result.add_row(row_result)
    return table_result


``` sql
SELECT title, IMDb-score FROM movies;
```

In [115]:
print(project(movies, [0, 2]))

+------------------------+------------+
|         title          | IMDb-score |
+------------------------+------------+
|    The Dark Knight     |    9.0     |
|       Fight Club       |    8.8     |
|         Matrix         |    8.7     |
|       Inception        |    8.8     |
|      Interstellar      |    8.7     |
| Avengers: Infinity War |    8.4     |
+------------------------+------------+


``` sql
SELECT title FROM movies;
```

In [116]:
print(project(movies, [0]))

+------------------------+
|         title          |
+------------------------+
|    The Dark Knight     |
|       Fight Club       |
|         Matrix         |
|       Inception        |
|      Interstellar      |
| Avengers: Infinity War |
+------------------------+


## Operation: Select

In [117]:
def select(table: PrettyTable, phi: Callable[[list], bool]) -> PrettyTable:
    table_result = PrettyTable()
    table_result.field_names = table.field_names
    for row in table.rows: 
        if phi(row):
            table_result.add_row(row)
    return table_result 

``` sql
SELECT * FROM movies
WHERE IMDb-score >= 8.8;
```

In [118]:
print(select(movies, lambda row: row[2] >= '8.8'))

+-----------------+------+------------+
|      title      | year | IMDb-score |
+-----------------+------+------------+
| The Dark Knight | 2008 |    9.0     |
|    Fight Club   | 1999 |    8.8     |
|    Inception    | 2010 |    8.8     |
+-----------------+------+------------+


``` sql
SELECT * FROM movies
WHERE year > 2010;
```

In [119]:
print(select(movies, lambda row: row[1] >= '2010'))

+------------------------+------+------------+
|         title          | year | IMDb-score |
+------------------------+------+------------+
|       Inception        | 2010 |    8.8     |
|      Interstellar      | 2014 |    8.7     |
| Avengers: Infinity War | 2018 |    8.4     |
+------------------------+------+------------+


``` sql
SELECT * FROM movies
WHERE year > 2010
AND IMDb-score >= 8.5;
```

In [120]:
print(select(movies, lambda row: row[1] >= '2010' and row[2] >= '8.5'))

+--------------+------+------------+
|    title     | year | IMDb-score |
+--------------+------+------------+
|  Inception   | 2010 |    8.8     |
| Interstellar | 2014 |    8.7     |
+--------------+------+------------+


## Operation: Cartesian-product

In [121]:
def cartesian_product(table_1: PrettyTable, table_2: PrettyTable) -> PrettyTable:
    table_result = PrettyTable()
    table_result.field_names = table_1.field_names + table_2.field_names
    for row1 in table_1.rows:
        for row2 in table_2.rows:
            new_row = row1 + row2
            table_result.add_row(new_row)
    return table_result

``` sql
SELECT * FROM movies, actors;
```

In [122]:
print(cartesian_product(movies, actors))

+------------------------+------+------------+------------+-----------+
|         title          | year | IMDb-score | first_name | last_name |
+------------------------+------+------------+------------+-----------+
|    The Dark Knight     | 2008 |    9.0     | Christian  |    Bale   |
|    The Dark Knight     | 2008 |    9.0     |    Bred    |    Pitt   |
|    The Dark Knight     | 2008 |    9.0     |   Meryl    |   Streep  |
|       Fight Club       | 1999 |    8.8     | Christian  |    Bale   |
|       Fight Club       | 1999 |    8.8     |    Bred    |    Pitt   |
|       Fight Club       | 1999 |    8.8     |   Meryl    |   Streep  |
|         Matrix         | 1999 |    8.7     | Christian  |    Bale   |
|         Matrix         | 1999 |    8.7     |    Bred    |    Pitt   |
|         Matrix         | 1999 |    8.7     |   Meryl    |   Streep  |
|       Inception        | 2010 |    8.8     | Christian  |    Bale   |
|       Inception        | 2010 |    8.8     |    Bred    |    P

## Operation: project, select and cartesian-product together

``` sql
SELECT title, year 
FROM movies
WHERE year >= 2012;
```

In [123]:
T1 = select(movies, lambda row: row[1] >= '2014')
T2 = project(T1, [0, 1])
print(T2)

+------------------------+------+
|         title          | year |
+------------------------+------+
|      Interstellar      | 2014 |
| Avengers: Infinity War | 2018 |
+------------------------+------+


``` sql
SELECT title, IMDb-score
FROM movies
WHERE IMDb=score >= 8.3
AND IMDb-score < 8.8;
```

In [124]:
T1 = select(movies, lambda row: row[2] >= '8.3' and row[2] < '8.8')
T2 = project(T1, [0, 2])
print(T2)

+------------------------+------------+
|         title          | IMDb-score |
+------------------------+------------+
|         Matrix         |    8.7     |
|      Interstellar      |    8.7     |
| Avengers: Infinity War |    8.4     |
+------------------------+------------+


### Creating new tables

In [125]:
movies = PrettyTable()
movies.field_names = ['m_id', 'title', 'year', 'score']
movies.add_row(['1', 'The Dark Night', '2008', '9.0'])
movies.add_row(['2', 'Inception', '2010', '8.8'])
movies.add_row(['3', 'Fight Club', '1999', '8.8'])
movies.add_row(['4', 'Titanic', '1997', '7.9'])

actors = PrettyTable()
actors.field_names = ['p_id', 'name']
actors.add_row(['1', 'Christian Bale'])
actors.add_row(['2', 'Brad Pitt'])
actors.add_row(['3', 'Leonardo DiCaprio'])
actors.add_row(['4', 'Kate Winslet'])

cast = PrettyTable()
cast.field_names = ['movie_id', 'person_id']
cast.add_row(['1', '1']) 
cast.add_row(['3', '2']) 
cast.add_row(['2', '3']) 
cast.add_row(['4', '3']) 
cast.add_row(['4', '4']) 

director = PrettyTable()
director.field_names = ['name', 'movie_title']
director.add_row(['Christopher Nolan', 'Interstellar'])
director.add_row(['Christopher Nolan', 'The Dark Night'])
director.add_row(['James Cameron', 'Titanic'])
director.add_row(['James Cameron', 'Avatar'])

print(movies)
print(actors)
print(cast)

+------+----------------+------+-------+
| m_id |     title      | year | score |
+------+----------------+------+-------+
|  1   | The Dark Night | 2008 |  9.0  |
|  2   |   Inception    | 2010 |  8.8  |
|  3   |   Fight Club   | 1999 |  8.8  |
|  4   |    Titanic     | 1997 |  7.9  |
+------+----------------+------+-------+
+------+-------------------+
| p_id |        name       |
+------+-------------------+
|  1   |   Christian Bale  |
|  2   |     Brad Pitt     |
|  3   | Leonardo DiCaprio |
|  4   |    Kate Winslet   |
+------+-------------------+
+----------+-----------+
| movie_id | person_id |
+----------+-----------+
|    1     |     1     |
|    3     |     2     |
|    2     |     3     |
|    4     |     3     |
|    4     |     4     |
+----------+-----------+


``` sql
SELECT movies.title, persons.name
FROM movies, persons, cast
WHERE movies.id = cast.movie_id
AND persons.id = cast.person_id;
```

In [126]:
T1 = cartesian_product(movies, actors)
T2 = cartesian_product(T1, cast)
T3 = select(T2, lambda row: row[0] == row[6] and row[4] == row[7])
T4 = project(T3, [1, 5])
print(T4)

+----------------+-------------------+
|     title      |        name       |
+----------------+-------------------+
| The Dark Night |   Christian Bale  |
|   Inception    | Leonardo DiCaprio |
|   Fight Club   |     Brad Pitt     |
|    Titanic     | Leonardo DiCaprio |
|    Titanic     |    Kate Winslet   |
+----------------+-------------------+


## Operation: Joins

### Single-column equi-join

In [127]:
def join(table_1: PrettyTable, table_2: PrettyTable, j1: int, j2: int) -> PrettyTable:
    table_result = PrettyTable()
    table_result.field_names = table_1.field_names + table_2.field_names
    for row1 in table_1.rows:
        for row2 in table_2.rows:
            if row1[j1] == row2[j2]:
                new_row = row1 + row2
                table_result.add_row(new_row)
    return table_result 

``` sql
SELECT * FROM movies, cast
WHERE movies.m_id = cast.movie_id;
```

In [128]:
print(join(movies, director, 1, 1))

+------+---------+------+-------+---------------+-------------+
| m_id |  title  | year | score |      name     | movie_title |
+------+---------+------+-------+---------------+-------------+
|  4   | Titanic | 1997 |  7.9  | James Cameron |   Titanic   |
+------+---------+------+-------+---------------+-------------+
